# Задание **Pro**

Преобразуйте отношение в таблице к 1 НФ, 2 НФ, 3 НФ

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.DataFrame(columns=['Full Names','Physical Address','Movies rented','Salutation'],data=[['Janet Jones','First Street Plot N0 4', 'Titanic, The girl next door', 'Ms.'],
                                                                                             ['Robert Phil','3rd Street 34', 'Blade 3, Casablanca','Mr.'],
                                                                                             ['Robert Phil','5th Avenue', 'Titanic','Mr.']])
df

,Full Names,Physical Address,Movies rented,Salutation
0,Janet Jones,First Street Plot N0 4,"Titanic, The girl next door",Ms.
1,Robert Phil,3rd Street 34,"Blade 3, Casablanca",Mr.
2,Robert Phil,5th Avenue,Titanic,Mr.


# Разбор таблицы

Данная таблица является таблицей видеопроката, которую нужно преобразовать сначала в **1НФ**, после во **2НФ**, а затем уже в **3НФ**.

Таблица состоит из:
- Фамилия Имя                      (**Full Names**)
- Адрес (**Physical Address**)
- Фильмы, которые взяты в прокат (**Movies rented**)
- Обращение (**Salutation**)

Как видно есть люди с **одинаковым** именем, но **разными** адресами. То есть это разные люди.

И для каждого человека может быть несколько фильмов. Как в первой строке: **Titanic**, **The girl next door**

# 1 НФ

Данная таблица не является 1НФ, потому что она должна включать в себя атомарность, данные по столбцам и по строкам должны быть максимально простыми, чтобы их нельзя было разделить.

Видно, что есть несколько фильмов, которые были взяли в прокат, одним человеком. А должны быть разные строки.

In [ ]:
# Можно воспользоваться методом .str.split() по сепоратору запятая, чтобы разделить фильмы

df['Movies rented'] = df['Movies rented'].str.split(',')

# Посмотрим на результат
df 

,Full Names,Physical Address,Movies rented,Salutation
0,Janet Jones,First Street Plot N0 4,"[Titanic, The girl next door]",Ms.
1,Robert Phil,3rd Street 34,"[Blade 3, Casablanca]",Mr.
2,Robert Phil,5th Avenue,[Titanic],Mr.


Получился список строк. В котором фильмы взятые в прокат одним человеком.

In [ ]:
# Начиная с версии Pandas 0.25.0 можно воспользоваться функцией explode()
# С помощью которой можно разделить указанные фильмы.

df = df.explode('Movies rented')
df

,Full Names,Physical Address,Movies rented,Salutation
0,Janet Jones,First Street Plot N0 4,Titanic,Ms.
0,Janet Jones,First Street Plot N0 4,The girl next door,Ms.
1,Robert Phil,3rd Street 34,Blade 3,Mr.
1,Robert Phil,3rd Street 34,Casablanca,Mr.
2,Robert Phil,5th Avenue,Titanic,Mr.


Теперь данные не делими. Преобразовали таблицу в **1НФ**.

# 2 НФ

Но данная таблица не является 2НФ. Потому что должен быть основной ключ - это Имя и адрес. Или создать ключ - **ID**.

Так как имя может совпадать как в данном случаи и разные адреса.

Во 2НФ основной ключ должен встречаться только 1 раз.

Разделим данную таблицу. В текущей останется все кроме фильмов. 

И еще одна таблица с фильмами

In [ ]:
df_name = df.drop('Movies rented', axis=1)

# Посмотрим на результат
df_name

,Full Names,Physical Address,Salutation
0,Janet Jones,First Street Plot N0 4,Ms.
0,Janet Jones,First Street Plot N0 4,Ms.
1,Robert Phil,3rd Street 34,Mr.
1,Robert Phil,3rd Street 34,Mr.
2,Robert Phil,5th Avenue,Mr.


Видно что есть **дубликаты**

In [ ]:
# С помощью метода drop_duplicates убираем дубликаты
df_name = df_name.drop_duplicates()

# Посмотрим на результат
df_name

,Full Names,Physical Address,Salutation
0,Janet Jones,First Street Plot N0 4,Ms.
1,Robert Phil,3rd Street 34,Mr.
2,Robert Phil,5th Avenue,Mr.


Еще одна таблица для **фильмов**

In [ ]:
df_movies = df['Movies rented']
df_movies

0                Titanic
0     The girl next door
1                Blade 3
1             Casablanca
2                Titanic
Name: Movies rented, dtype: object

In [ ]:
# Преобразуем данный столбец с фильмами в DataFrame
df_movies = pd.DataFrame(df_movies)

# Посмотрим на результат
df_movies

,Movies rented
0,Titanic
0,The girl next door
1,Blade 3
1,Casablanca
2,Titanic


В данной таблице указан **ID** человека и фильм который взял в прокат.

# 3 НФ

В **3** НФ у нас не должно быть связей между не ключевыми столбцами.

Есть прямая связь между **Именем** и **Обращением**.

Столбец Обращение(**Salutation**) нужно вынести в отдельную таблицу.

In [ ]:
# Сделаем это так же как и во 2НФ
df_salutation = df_name['Salutation']
df_salutation = pd.DataFrame(df_salutation)

# Посмотрим на результат
df_salutation

,Salutation
0,Ms.
1,Mr.
2,Mr.


И в базе имен **df_name** соответственно удалим данный столбец

In [ ]:
df_name = df_name.drop('Salutation', axis=1)
df_name

,Full Names,Physical Address
0,Janet Jones,First Street Plot N0 4
1,Robert Phil,3rd Street 34
2,Robert Phil,5th Avenue


То есть **3 НФ** будет состоять из трех таблиц:
- Фильмы (df_movies)
- Имена с адресами (df_name)
- Обращение (df_salutation)

## Результат

In [ ]:
df_movies

,Movies rented
0,Titanic
0,The girl next door
1,Blade 3
1,Casablanca
2,Titanic


In [ ]:
df_name

,Full Names,Physical Address
0,Janet Jones,First Street Plot N0 4
1,Robert Phil,3rd Street 34
2,Robert Phil,5th Avenue


In [ ]:
df_salutation

,Salutation
0,Ms.
1,Mr.
2,Mr.
